### 01 Generate SQL Database from NYC Restaurant Inspection Data

New York City publishes its latest restaurant inspection [results](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j) daily. The dataset contains results for up to the past three years and with over 200k rows, this data is great for experimenting with SQL.

Objective: Preprocess data into a SQL database with multiple tables in preparation for experimenting with advanced SQL techniques

In [1]:
import pandas as pd
from pathlib import Path
import sqlite3

Let's import the full dataset into pandas and split it into separate tables before adding them to an SQL database. This will enable experimenting with advanced SQL table joins in order to extract information of interest in the follow-on notebooks. We'll also do a little data cleaning before exporting to the SQL database.

In [2]:
# Load full dataset
all_data = pd.read_csv(Path('assets/DOHMH_New_York_City_Restaurant_Inspection_Results.csv'))

# Inspect
print(all_data.shape)
all_data.columns

(208139, 32)


Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA', 'Location Point', 'Zip Codes', 'Community Districts',
       'Borough Boundaries', 'City Council Districts', 'Police Precincts'],
      dtype='object')

In [3]:
# Rename columns
all_data = all_data.rename(columns={'INSPECTION DATE': 'INSPECDATE',
                                    'INSPECTION TYPE': 'INSPECTYPE',
                                    'CUISINE DESCRIPTION': 'CUISDESC',
                                    'VIOLATION CODE': 'VIOLCODE',
                                    'VIOLATION DESCRIPTION': 'VIOLDESC'})
# Process datatypes
all_data['ZIPCODE'] = all_data['ZIPCODE'].fillna(0).astype({'ZIPCODE': 'int32'})
all_data['INSPECDATE'] = pd.to_datetime(all_data['INSPECDATE'], errors='coerce')

# Inspect
print(all_data.dtypes)
all_data.tail(3)

CAMIS                              int64
DBA                               object
BORO                              object
BUILDING                          object
STREET                            object
ZIPCODE                            int32
PHONE                             object
CUISDESC                          object
INSPECDATE                datetime64[ns]
ACTION                            object
VIOLCODE                          object
VIOLDESC                          object
CRITICAL FLAG                     object
SCORE                            float64
GRADE                             object
GRADE DATE                        object
RECORD DATE                       object
INSPECTYPE                        object
Latitude                         float64
Longitude                        float64
Community Board                  float64
Council District                 float64
Census Tract                     float64
BIN                              float64
BBL             

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISDESC,INSPECDATE,ACTION,...,Census Tract,BIN,BBL,NTA,Location Point,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
208136,50088694,STATEN ISLAND JUICEBAR,Staten Island,3231,RICHMOND AVENUE,10312,9174499959,"Juice, Smoothies, Fruit Salads",2020-03-03,Violations were cited in the following area(s).,...,14607.0,5073738.0,5.055330e+09,SI54,NaN,NaN,NaN,NaN,NaN,NaN
208137,50117776,SEEYAMANANA,Manhattan,49,WEST 27 STREET,10001,2126894002,Tapas,2022-12-02,Violations were cited in the following area(s).,...,5800.0,1015676.0,1.008290e+09,MN13,NaN,NaN,NaN,NaN,NaN,NaN
208138,50111537,MOCHINUT,Manhattan,1001,AVENUE OF THE AMERICAS,10018,7188888868,Other,2022-07-05,Violations were cited in the following area(s).,...,10900.0,1015255.0,1.008130e+09,MN17,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create table for inspection results
inspections = all_data[['INSPECDATE', 'INSPECTYPE', 'CAMIS', 'SCORE', 'GRADE', 
                        'VIOLCODE']]
inspections.tail()

,INSPECDATE,INSPECTYPE,CAMIS,SCORE,GRADE,VIOLCODE
208134,2019-07-03,Cycle Inspection / Initial Inspection,50014500,29.0,NaN,04J
208135,2022-04-25,Cycle Inspection / Initial Inspection,50098771,0.0,NaN,02B
208136,2020-03-03,Pre-permit (Operational) / Initial Inspection,50088694,8.0,A,06C
208137,2022-12-02,Pre-permit (Operational) / Initial Inspection,50117776,43.0,NaN,04A
208138,2022-07-05,Pre-permit (Operational) / Initial Inspection,50111537,45.0,NaN,04J


In [5]:
# Create table for restaurant info
restaurants = all_data[['CAMIS', 'DBA', 'CUISDESC', 'ZIPCODE', 'BORO', 'Latitude', 'Longitude']]
restaurants.tail()

,CAMIS,DBA,CUISDESC,ZIPCODE,BORO,Latitude,Longitude
208134,50014500,CROWN FRIED CHICKEN,Chicken,11221,Brooklyn,40.695320,-73.920328
208135,50098771,ANTOJITOS JUICE BAR,Latin American,10456,Bronx,40.836157,-73.902725
208136,50088694,STATEN ISLAND JUICEBAR,"Juice, Smoothies, Fruit Salads",10312,Staten Island,40.560393,-74.169764
208137,50117776,SEEYAMANANA,Tapas,10001,Manhattan,40.744904,-73.989740
208138,50111537,MOCHINUT,Other,10018,Manhattan,40.751835,-73.986310


In [6]:
# Create table for violation info
violations = all_data[['VIOLCODE', 'VIOLDESC']].dropna()
violations.head()

,VIOLCODE,VIOLDESC
39,02B,Hot food item not held at or above 140º F.
66,22G,Sale or use of certain expanded polystyrene it...
83,02B,Hot food item not held at or above 140º F.
91,10J,Hand wash sign not posted
95,02B,Hot food item not held at or above 140º F.


With the data somewhat cleaned and the tables generated, lets load everything into an SQL database.

In [7]:
conn = sqlite3.connect('nyc_inspections.db')

tables = [inspections, restaurants, violations]
names = ['inspections', 'restaurants', 'violations']

for table, name in zip(tables, names):
    table.to_sql(name, con=conn, index=False, if_exists='replace')

In [8]:
%load_ext sql
%sql sqlite:///nyc_inspections.db

In [9]:
%%sql
SELECT name FROM sqlite_schema;

 * sqlite:///nyc_inspections.db
Done.


name
inspections
restaurants
violations


In [10]:
%%sql
SELECT * FROM inspections
LIMIT 5;

 * sqlite:///nyc_inspections.db
Done.


INSPECDATE,INSPECTYPE,CAMIS,SCORE,GRADE,VIOLCODE
1900-01-01 00:00:00,None,50125332,None,None,None
1900-01-01 00:00:00,None,50113608,None,None,None
1900-01-01 00:00:00,None,50132947,None,None,None
1900-01-01 00:00:00,None,50131641,None,None,None
1900-01-01 00:00:00,None,50107034,None,None,None


In [11]:
%%sql
SELECT * FROM restaurants
LIMIT 5;

 * sqlite:///nyc_inspections.db
Done.


CAMIS,DBA,CUISDESC,ZIPCODE,BORO,Latitude,Longitude
50125332,None,None,11103,Queens,40.766568442385,-73.905123532108
50113608,BIRDIES HOT CHICKEN,None,11211,Brooklyn,40.71190313079,-73.941113998372
50132947,None,None,11235,Brooklyn,40.584915043468,-73.951033402941
50131641,None,None,11385,Queens,40.700198634374,-73.90680880349
50107034,POPYE'S,None,11691,Queens,40.596422966153,-73.754162918486


In [12]:
%%sql
SELECT * FROM violations
LIMIT 5;

 * sqlite:///nyc_inspections.db
Done.


VIOLCODE,VIOLDESC
02B,Hot food item not held at or above 140º F.
22G,Sale or use of certain expanded polystyrene items restricted
02B,Hot food item not held at or above 140º F.
10J,Hand wash sign not posted
02B,Hot food item not held at or above 140º F.


Database and tables look ready for SQL analysis. Onto the next notebook!